## Лекция №2

### Восстановление пропущенных значений

В таблице могут быть пустые ячейки (с пропущенными значениями) и/или с некорректными значениями.

In [70]:
# Необходимые пакеты
import numpy as np
import pandas as pd

In [100]:
df = pd.read_csv('data/table_0201.csv')
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
6,Алексеева,NaN,NaN,NaN,NaN,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


#### Что можно/нужно сделать?

Если пропущенных значений много -> удалить объект (строку)

In [101]:
df_new = df.drop([6])
df_new

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Чтобы удалить строку в "оригинальной" таблице, необходимо удалять с параметром inplace=True

In [102]:
df.drop([6], inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
7,Андреев,1.0,NaN,NaN,3.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Удалим объекты (строки), где количество значимых (не NaN) признаков меньше 4-х

In [103]:
df.dropna(thresh=4, inplace=True)   # thresh=4, т.е. не меньше 4-х. Удалим объект Андреев (количество значимых признаков == 3)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде,Город
0,Иванов,1.0,172.0,107.0,3.0,NaN
1,Запеканка,NaN,185.0,71.0,-4.0,NaN
2,Ватрушкина,0.0,168.0,666.0,2.0,NaN
3,Ололоева,0.0,NaN,85.0,1.0,NaN
4,Сидорова,0.0,165.0,15.0,2.0,NaN
5,Петров,1.0,172.0,71.0,4.0,NaN
8,Новикова,0.0,NaN,53.0,4.0,NaN


Как мы видим признак (столбец) "Город" не несет никакой "смысловой нагрузки". Его надо удалить.

In [104]:
df.dropna(thresh=1, axis="columns", inplace=True)    # или axis=1 - это столбцы, axis=0 - это строки (по умолчанию)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,-4.0
2,Ватрушкина,0.0,168.0,666.0,2.0
3,Ололоева,0.0,NaN,85.0,1.0
4,Сидорова,0.0,165.0,15.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,NaN,53.0,4.0


Поправим некорректные значения. Вес 666 кг это уже слишком (а 15 кг слишком маленький). И место не может быть меньше < 1.

In [105]:
df.loc[(df['Вес'] > 200) | (df['Вес'] < 20), 'Вес'] = None

In [106]:
df.loc[(df['Место на олимпиаде'] < 1), 'Место на олимпиаде'] = None
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,NaN,2.0
3,Ололоева,0.0,NaN,85.0,1.0
4,Сидорова,0.0,165.0,NaN,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,NaN,53.0,4.0


Теперь можно заняться восстановлением "пропущенных" значений (NaN). Вот некоторые варианты замены:
- Среднее арифметическое значение
- Медиана
- Мода

Заменим пропущенные значения в признаке "Рост" на среднее арифметическое признака.

In [107]:
df['Рост'].fillna(df['Рост'].mean(), inplace=True) #
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,NaN,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,NaN,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Пропущенные значения в признаке "Вес" заменим на моду (самое часто встречающее значение == 71)

In [114]:
df['Вес'].fillna(df['Вес'].mode()[0], inplace=True) # mode()[0] - первая мода, т.к. их может быть несколько
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,NaN
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Для места на олимпиаде выберим медиану.

In [123]:
# ! Медиану округлим до целого, т.к. место может быть только целым )))
df['Место на олимпиаде'].fillna(round(df['Место на олимпиаде'].median()), inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,NaN,185.0,71.0,2.0
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0


Усложним чуть-чуть задачу. Признак "Пол" заполним значением с определенной вероятностью.
Значение 1 (мужской пол), всего 2 из 6, итого вероятность = 2/6.
Значение 0 (женский пол), всего 4 из 6, итого вероятность = 4/6

In [127]:
count_gender = df['Пол'].count()      # всего значений в признаке "Пол"

In [137]:
male_weight = len(df[df['Пол'] == 1]) / count_gender      # вероятность для "мужского пола"

In [138]:
female_weight = len(df[df['Пол'] == 0]) / count_gender    # вероятность для "женского пола"

In [157]:
import random
df['Пол'].fillna(random.choices([1, 0], weights=[male_weight,female_weight])[0], inplace=True)
df

,Студент,Пол,Рост,Вес,Место на олимпиаде
0,Иванов,1.0,172.0,107.0,3.0
1,Запеканка,0.0,185.0,71.0,2.0
2,Ватрушкина,0.0,168.0,71.0,2.0
3,Ололоева,0.0,172.4,85.0,1.0
4,Сидорова,0.0,165.0,71.0,2.0
5,Петров,1.0,172.0,71.0,4.0
8,Новикова,0.0,172.4,53.0,4.0
